# Intensity dependant measurement analysis
* This notebook is for analysis of intensity dependant data taken from the PLQe imaging setup. 
* Assumes pre processing has already been done
    
### Some instructions: 
* Please run each cell in order, following the instructions
    * Press cntrl+enter to run a cell
* When prompted enter information into the feilds and press enter
* If you go back and re run a cell, that's fine but you have to run every cell after that again, and in order

This might break if all these files are not there! If you want to edit this code definately go ahead, a bunch of the functions live in seperate .py files, BUT please make a copy!!!

In [ ]:
# Some imports:
from external_imports import *
from image_process import *
from general_data_process import *
from intensity_dependant import *

# Plot parameters
import matplotlib
matplotlib.rcParams['font.size'] = 12
matplotlib.rcParams['font.family'] = "Century Gothic"

## Core process
* To do anything we need the PLQE maps, so let's make those first
* Follow these steps:

#### STEP 1: Determine what data we have
* Provide a path to the data folder and folder you would like to save everything in
    * Just copy and paste paths as raw strings (r"path\to\data") into approriate fields 
* Then, for each substrate, provide the bandgap in the input fields
* If you have alreay saved the PLQE maps before and are just re-running this book just run this cell to remind the kernal where all the files live

In [ ]:
datapath = r"E:\Data\EL_setup\EL_cam_paper\final_set\cells"
savepath = r"E:\Data\EL_setup\EL_cam_paper\final_set\Plots\new_processed_data"
if not os.path.isdir(savepath):
    os.makedirs(savepath)

path_db = path_process(datapath)

bandgaps = {}
for key in path_db.keys():
    if key.lower() != 'white':
        string = "What's the bandgap of "+key+" in eV?: "
        bandgap = float(input(string))
        bandgaps[key] = bandgap

#### STEP 2: Calculate PLQEs
* Now just run the following to save the PLQE maps. 

In [ ]:
whiteparamsfile = f"{datapath}\\white_params.csv"  # defines location of white ref parameter file
possible_vsweep_folders = ['vsweep', 'vsweep_f', 'vsweep_b']  # For getting a flux at one sun
int_dep_folders = ['oc', 'sc'] # Folders of interest

for key in path_db.keys():
    bandgap = bandgaps[key]
    for pix in path_db[key]: 
        whitefilename = f"{datapath}\\{key}\\{pix}\\white.npy"
        
        # extracting flux at one sun (if avaliable)
        flux1sun = 0
        for vsweep_folder in possible_vsweep_folders:
            if os.path.isdir(f"{datapath}\\{key}\\{pix}\\{vsweep_folder}"):
                flux1sun = float(find_npy(f"{datapath}\\{key}\\{pix}\\{vsweep_folder}")[0].split('_')[2])
                break
        # save PLQEs for oc and sc
        for int_dep_folder in int_dep_folders:
            datapath_i = f"{datapath}\\{key}\\{pix}\\{int_dep_folder}"
            savepath_i = f"{savepath}\\{key}\\{pix}"
            if os.path.isdir(datapath_i):
                save_PLQE(datapath_i, savepath_i, whitefilename, whiteparamsfile, bandgap, 
                          flux1sun=flux1sun, savename=int_dep_folder)

* Plots (png files for easy viewing):

In [ ]:
# Run this to save the images
for key in path_db.keys():
    for pix in path_db[key]:
        for int_dep_folder in int_dep_folders:
            path_i = f"{savepath}\\{key}\\{pix}\\PLQE_{int_dep_folder}"
            if os.path.isdir(path_i):
                if not os.path.isdir(f"{path_i}\\pngs"):
                    os.makedirs(f"{path_i}\\pngs")

                filenames = find_npy(path_i)

                def process_one_file(filename):
                    savefile_name = '_'.join(filename.split('_')[:-1])+'.png'
                    num_sun = float(filename.split('_')[1])
                    
                    fig, ax = plt.subplots()
                    imarr = np.load(f"{path_i}\\{filename}")
                    im = ax.imshow(imarr*100, cmap="inferno", vmin=np.mean(imarr*100)-3*np.std(imarr*100),
                                                              vmax=np.mean(imarr*100)+3*np.std(imarr*100))
                    
                    ax.set_title(f'num_sun = {num_sun}')
                    cbar = plt.colorbar(im,pad=0.01,aspect=20)
                    cbar.set_label('PLQE (%)', rotation=270,labelpad=25)

                    scalebar_size = 0.1 # cm
                    scalebar_offset = 10 # pix

                    scalebar_width_pix = 0.1*pixels_per_cm
                    scalebar_height_pix = scalebar_width_pix/10
                    scalbar_y_position = imarr.shape[0]-scalebar_offset-scalebar_height_pix

                    text_x = scalebar_offset + scalebar_width_pix/2
                    text_y = scalbar_y_position

                    scalebar = patches.Rectangle((scalebar_offset,scalbar_y_position),scalebar_width_pix,scalebar_height_pix,linewidth=1,edgecolor='k',facecolor='white')
                    ax.add_patch(scalebar)
                    txt = plt.text(text_x,text_y,"1 mm", color = "white", ha='center', va='bottom', fontweight='bold')
                    txt.set_path_effects([PathEffects.withStroke(linewidth=1.2, foreground='k')])

                    plt.axis('off')

                    plt.savefig(f"{path_i}\\pngs\\{savefile_name}", dpi=300)
                    plt.close('all')
                Parallel(n_jobs=num_cores)(delayed(process_one_file)(filename) for filename in filenames)

## Different analysis modules:
* The following will be different cells that do different bits of analysis, just run the ones you want

### Reconstructed JV (averaged) and sun's Voc
* Calculate Psudo J/V and Sun's Voc JV averaging over whole cropped image
* Saves data in pix folder as a csv, as well as a png image 
* Uncomment plt.show() to see the plots here

In [ ]:
for key in path_db.keys():
    for pix in path_db[key]:     
        path_i = f"{savepath}\\{key}\\{pix}"
        bandgap = bandgaps[key]
        voc_rad0 = vocradf(bandgap)

        QFLSs, Js, bias, fluxes, num_suns = average_recon_jv(path_i, voc_rad0)
        
        suns_voc_V = []
        with open(f"{datapath}\\{key}\\{pix}\\oc\\source_meter.csv",'r') as file:
            reader = csv.reader(file)
            for row in reader:
                suns_voc_V.append(float(row[0]))
                
        suns_voc_V = np.array(sorted([abs(i) for i in suns_voc_V]))
        with open(f"{path_i}\\reconstructed_JV_ave.csv",'w',newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["QFLS(ev)", "measured voltage (V)", "reconstructed current(J/Jsc)"])
            writer.writerows(zip(QFLSs, suns_voc_V, Js))

Plots:

In [ ]:
for key in path_db.keys():
    for pix in path_db[key]: 
        path_i = f"{savepath}\\{key}\\{pix}"
        QFLSs, suns_voc_V, Js = ([],[],[])
        with open(f"{path_i}\\reconstructed_JV_ave.csv",'r') as file:
            reader = csv.reader(file)
            next(reader)
            for row in reader:
                QFLSs.append(float(row[0]))
                suns_voc_V.append(float(row[1]))
                Js.append(float(row[2]))
        
        QFLSs = np.array(QFLSs)
        suns_voc_V = np.array(suns_voc_V)
        Js = np.array(Js)
        
        
        plt.scatter(-QFLSs, Js, color='k', label = 'Reconstructed')
        plt.plot(-suns_voc_V, Js, color='r', label = "Sun's Voc measurement")
        
        plt.xlim((-1.3,-0.3))      
        plt.ylim((-1,1))
        plt.axhline(0,color='k')
        
        plt.legend(frameon=False)
        plt.xlabel("Voltage (V)")
        plt.ylabel("J/J$_{sc}$")
        plt.title(f"{key}, Pixel {pix}")
        
        plt.savefig(f"{path_i}\\reconstructed_JV_ave.png", dpi=320)
#         plt.show()
        plt.close('all')


### Intensity dependant QFLS maps
* Creates maps of QFLS for each optical bias
* Saves in new folder in pix folder
* Draws from PLQE_oc folder, so that needs to be present

In [ ]:
for key in path_db.keys():
    for pix in path_db[key]:  
        for bias in ['oc', 'sc']:
            path_i = f"{savepath}\\{key}\\{pix}"
            bandgap = bandgaps[key]
            voc_rad0 = vocradf(bandgap)

            intsweep_QFLS(path_i, voc_rad0, bias)

Plots (saved in 'png' folder inside 'QFLS_int_oc):

In [ ]:
# Run this to save the images
for key in path_db.keys():
    for pix in path_db[key]:
        path_i = f"{savepath}\\{key}\\{pix}\\QFLS_int_oc"
        if not os.path.isdir(f"{path_i}\\pngs"):
            os.makedirs(f"{path_i}\\pngs")
        
        filenames = find_npy(path_i)

        def process_one_file(filename):
            savefile_name = '_'.join(filename.split('_')[:-1])+'.png'
            fig, ax = plt.subplots()
            
            num_suns = float(filename.split('_')[1])
            jbyjsc = 1-num_suns

            imarr = np.load(f"{path_i}\\{filename}")
            im = ax.imshow(imarr, cmap="inferno")#, vmin=1)
            ax.set_title('J/J$_{sc}$ = '+str(jbyjsc))
            cbar = plt.colorbar(im,pad=0.01,aspect=20)
            cbar.set_label('V$_{oc}$ (V)', rotation=270,labelpad=25)

            scalebar_size = 0.1 # cm
            scalebar_offset = 10 # pix

            scalebar_width_pix = 0.1*pixels_per_cm
            scalebar_height_pix = scalebar_width_pix/10
            scalbar_y_position = imarr.shape[0]-scalebar_offset-scalebar_height_pix

            text_x = scalebar_offset + scalebar_width_pix/2
            text_y = scalbar_y_position

            scalebar = patches.Rectangle((scalebar_offset,scalbar_y_position),scalebar_width_pix,scalebar_height_pix,linewidth=1,edgecolor='k',facecolor='white')
            ax.add_patch(scalebar)
            txt = plt.text(text_x,text_y,"1 mm", color = "white", ha='center', va='bottom', fontweight='bold')
            txt.set_path_effects([PathEffects.withStroke(linewidth=1.2, foreground='k')])

            plt.axis('off')

            plt.savefig(f"{path_i}\\pngs\\{savefile_name}", dpi=300)
            plt.close('all')
        Parallel(n_jobs=num_cores)(delayed(process_one_file)(filename) for filename in filenames)

### Ideality factor map
* Calculates map of ideality factor 
* Draws from QFLS_int_oc, so relies on that being present
* .npy and png saved in pix folder

In [ ]:
for key in path_db.keys():
    for pix in path_db[key]:
        path_i = f"{savepath}\\{key}\\{pix}"
        nid = get_idelity_map(path_i)
        np.save(f"{path_i}\\ideality", nid)
        
        fig, ax = plt.subplots()
        im = ax.imshow(nid, cmap="inferno", vmin=1, vmax=2)

        cbar = plt.colorbar(im,pad=0.01,aspect=20)
        cbar.set_label('n$_{id}$', rotation=270,labelpad=25)

        scalebar_size = 0.1 # cm
        scalebar_offset = 10 # pix

        scalebar_width_pix = 0.1*pixels_per_cm
        scalebar_height_pix = scalebar_width_pix/10
        scalbar_y_position = nid.shape[0]-scalebar_offset-scalebar_height_pix

        text_x = scalebar_offset + scalebar_width_pix/2
        text_y = scalbar_y_position

        scalebar = patches.Rectangle((scalebar_offset,scalbar_y_position),scalebar_width_pix,scalebar_height_pix,linewidth=1,edgecolor='k',facecolor='white')
        ax.add_patch(scalebar)
        txt = plt.text(text_x,text_y,"1 mm", color = "white", ha='center', va='bottom', fontweight='bold')
        txt.set_path_effects([PathEffects.withStroke(linewidth=1.2, foreground='k')])

        plt.axis('off')

        plt.savefig(f"{path_i}\\ideality.png", dpi=300)
        plt.close('all')
